**NOTE (09/02/24):** Just trying to put together the body of the simulation to make it work. Aiming to use Averill and Law's architecture of what a simulation looks like.


**NOTE (11/02/2024)What is different in version 2:** 
* Going to try and increase the number of flows going through the network from 1 to maybe 50. - Can get up to 9500 before the data limit in Jupyter is reached
* Going to implement a reporting function to collect data in each execution of the while loop. - DONE
* work on the departure event code in the event routine - not necessary
* work on the dictionary key problem with the key values - cannot think of a cleaner solution
* just work on cleaning up the code, making sure I have scoped all variables accordingly - not worrying too much about right now

**NOTE (13/02/2024):** 
* Need to work on generating future events within the events routine and not just in the initialisation routine.

## Import dependencies

In [205]:
import networkx as nx
import torch
import torch.nn as nn
import random
import time
import uuid
import bisect
import operator

## Constants and data containers

In [206]:
# Generation
GENERATOR = None
PATH = "generator_2.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 77

# Network & Flows
NETWORK = None
BLOCKED = None # count of flows blocked
SENT = None # count of flows that reached their destination
ADDRESSES = [i for i in range(1,22)]
WAVELENGTHS = 40

# Simulation
SIM_CLOCK = None 
MIN_FLOWS_SENT = 1e7
EVENTS_LIST = None

##completed_flows = []
#blocked_flows = []

## Classes

In [207]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [208]:
class Network:
    def __init__(self, graph, num_of_wavelengths):
        self.topology = graph
        self.__allocate_capacity__(num_of_wavelengths)
    
    
    def __allocate_capacity__(self, num_of_wavelengths):
        self.links = dict()
        for edge in self.topology.edges():
            node1, node2 = edge[0], edge[1] 
            self.links[(node1, node2)] = num_of_wavelengths 
       
    
    def __check_capacity__(self, node1, node2): 
        # NOTE: Need to add some exception handling because the node pair for the next hop may not correspond with the dictionary of links
        # instead will try it in the correct order, else just switch the arguments around
        try:
            return self.links[(node1, node2)] > 0
        except KeyError:
            return self.links[(node2, node1)] > 0
    
    
    def __use_capacity__(self, node1, node2):
        try: 
            self.links[(node1, node2)] -= 1
        except KeyError:
            self.links[(node2, node1)] -= 1

            
    def __release_capacity__(self, node1, node2):
        try:   
            self.links[(node1, node2)] += 1
        except KeyError:
            try:
                self.links[(node2, node1)] += 1
            except KeyError: # One of the nodes must be None
                pass # there is no link to release capacity for
                
        
        
    def push_flow(self, flow, type, node1, node2):
        # global BLOCKED
        if type == "HOP":
            self.__release_capacity__(flow.prev_node, flow.current_node)
            
        sufficient_capacity = self.__check_capacity__(node1, node2)
        if sufficient_capacity:
            self.__use_capacity__(node1, node2)
            next_event_type = flow.hop()
            return next_event_type, flow 
        else:
            # print out that Flow ID# has been blocked
            # BLOCKED += 1
            next_event_type = "BLOCKED"
            return next_event_type, flow
            
    def end_flow(self, flow, node1, node2):
        self.__release_capacity__(node1, node2)
        # print(f"Sim Clock: {SIM_CLOCK} seconds | Flow {flow.ID} has reached its destination, node {flow.dst}. Now leaving the network...")


    def find_route(self, src, dst):
        return nx.shortest_path(self.topology, source = src, target = dst)
    

In [209]:
class Flow:
    def __init__(self):
        # self.parent_net = network
        self.__create_flow__()
        
    def __generate_data__(self):
        random_noise = torch.randn((1, 2), device = DEVICE)
        generated_samples = GENERATOR(random_noise)
        generated_samples = generated_samples.cpu().detach().numpy()
        synthetic_dur, synthetic_size = generated_samples[0,0], generated_samples[0,1]
        return synthetic_dur, synthetic_size
        
    def __create_flow__(self):
        self.ID = str(uuid.uuid4())
        # self.status = "IDLE"
        self.dur, self.size = self.__generate_data__()
        random_addresses = random.sample(ADDRESSES, 2)
        self.src, self.dst = random_addresses[0], random_addresses[1]
        self.current_node = self.src
        self.prev_node = None
        
        # Calculate shortest path
        self.route = NETWORK.find_route(self.src, self.dst)
        
        # Removes the first node in the route which is the starting node for the flow path 
        self.route = self.route[1:] 
        
        # How long it will take to make each hop, used to schedule the next event time
        self.hop_time = self.dur / len(self.route) 

        
    def hop(self): # Pushes the flow into the network from src to dst/from one hop to the next
        self.status = "SENDING"
        self.prev_node = self.current_node 
        # print(f"Sim Clock: {SIM_CLOCK} seconds | Flow {self.ID} is moving from node {self.current_node} to node {self.route[0]}.")
        self.current_node = self.route.pop(0)
        
        if len(self.route) == 0:
            # print(f"Flow {self.ID} has reached its destination, node {self.dst}. Now leaving the network...")
            next_event_type = "DEPART"
        else:
            next_event_type = "HOP"
        
        return next_event_type

In [210]:
class Event:
    def __init__(self, event_time, event_type, flow):
        self.event_type = event_type
        self.event_time = event_time
        
        # Link a flow to a particular event that occurs, whether it arrives, hops or departs
        self.associated_flow = flow

## Functions

In [212]:
def schedule_event(event_time, event_type, associated_flow):
    global EVENTS_LIST
    
    # Create new event
    new_event = Event(event_time, event_type, associated_flow)
    
    # Create key with which the events list will be sorted; will be sorted by event time
    time_key = operator.attrgetter("event_time")
    
    # Inserts a new event into the Events list and maintains its sorted order
    bisect.insort(EVENTS_LIST, new_event, key=time_key) 


In [213]:
def initialise_simulation():
    global SIM_CLOCK, GENERATOR, NETWORK, BLOCKED, SENT, EVENTS_LIST
    
    EVENTS_LIST = []
    # Set simulation clock to 0
    SIM_CLOCK = 0
    # global sim_clock, generator, network, BLOCKED, SENT

    # Initialise the generator
    GENERATOR = Generator()
    GENERATOR.load_state_dict(torch.load(PATH))
    GENERATOR = GENERATOR.to(DEVICE)
    
    # Initialise the network
    G = nx.read_adjlist("UKnet", nodetype=int)
    NETWORK = Network(G, WAVELENGTHS)
    
    # Initialise statistical counters
    BLOCKED = 0
    SENT = 0
    
    # Adds 50 initial events to the events list - they will all be marked as hops 
    for i in range(9000):
        
        # Generate new flow
        new_flow = Flow()

        # Create and schedule the event
        schedule_event(event_time = i, event_type = "ARRIVAL", associated_flow = new_flow)


In [214]:
# Check the next event in the list and advance simulation clock
def timing_routine():
    global SIM_CLOCK
    # Popping the next event from the top of the list
    next_event = EVENTS_LIST.pop(0)
    
    # Accessing the time of the next scheduled event
    advanced_time = next_event.event_time
    
    # Advancing the simulation clock to the time of the next scheduled event
    SIM_CLOCK = advanced_time
    
    return next_event

In [215]:
def event_routine(event):
    global SIM_CLOCK, NETWORK, SENT, BLOCKED

    current_flow = event.associated_flow
    
    if event.event_type == "ARRIVAL" or event.event_type == "HOP":
        
        # Accessing the current node and next node in the flow path
        src, dst = current_flow.current_node, current_flow.route[0]
        
        # Allow the network to push the flow to the next node
        new_event_type, updated_flow = NETWORK.push_flow(current_flow, event.event_type, src, dst)
        
        if new_event_type != "BLOCKED": # if blocked, there's no new event to schedule and capacity in the previous link has been released.
            schedule_event(SIM_CLOCK+current_flow.hop_time, new_event_type, associated_flow = updated_flow)
        else:
            BLOCKED += 1
            
        
    else: # Departure event
        prev, current = current_flow.prev_node, current_flow.current_node
        
        # Release the capacity from the last hop 
        NETWORK.end_flow(current_flow, prev, current)
        
        SENT += 1

In [216]:
def report():
    blocked_flows = round(BLOCKED*100/(BLOCKED+SENT), 4)
    sent_flows = 100 - blocked_flows
    
    print(f"Total flows travelled through the network: {BLOCKED + SENT}")
    print(f"{sent_flows} % of all flows arrived successfully to its destination, {SENT} flows")
    print(f"{blocked_flows} % of all flows were blocked due to lack of capacity in a link, {BLOCKED} flows")
    

In [217]:
def main():
    print("Starting simulation...")
    initialise_simulation()
    
    start_time = time.time()
    # Run simulation until a certain amount of flows have been sent across the network
    while (SENT + BLOCKED) < MIN_FLOWS_SENT:
        
        next_event = timing_routine()
        # invoke timing routine - checking what the next event is and advancing simulation clock to that time
            # check the events list (I will be making the events list automatically sorted so that the next event time is at the front)
            # might make an Event class to store what exactly is going to happen at that point in time that stores the flow:
                # will it be an initial arrival, an intermediary hop, or a departure
                # if arrival - just make new HOP event to move to the next node
                # if hop - new event to move to the next node
                # if departure - then no new events added, just update statistical counters
    
        # invoke event routine - update system state, statistical counters, generate future events, add future events to events list
            # will invoke a specific function, either: arrival(), hop() or departure()
            # checking the link capacity
                # yes capacity - hops to next node
                # no capacity - blocked
            # add a new event of type arrival, hop or departure
        
    # compute and write report on estimates of interest
        # how many flows were sent in total
        # how many flows were blocked
        # how many flows reached their destination
    


In [218]:
# Main function for testing purposes
def main2():
    print("hello")
    print("Starting simulation...")
    initialise_simulation()
    while EVENTS_LIST:
        next_event = timing_routine()
        event_routine(next_event)
    report()


In [219]:
if __name__ == "__main__":
    main2()
    
# NOTE: At the end of the simulation

print(NETWORK.links)
print()

overcapacity = 0
overcapacity_links = []
for link in NETWORK.links:
    if NETWORK.links[link] > 40:
        overcapacity += 1
        overcapacity_links.append(link)
        
print(f"{overcapacity} links are over capacity")
print("The links are:")
for link in overcapacity_links:
    print(link, end=" ")


hello
Starting simulation...
Total flows travelled through the network: 9000
28.2222 % of all flows arrived successfully to its destination, 2540 flows
71.7778 % of all flows were blocked due to lack of capacity in a link, 6460 flows
{(1, 3): 40, (1, 2): 40, (1, 20): 40, (1, 4): 40, (1, 6): 40, (1, 8): 40, (1, 14): 40, (3, 2): 40, (3, 20): 40, (3, 19): 40, (2, 19): 40, (20, 9): 40, (20, 21): 40, (4, 19): 40, (4, 7): 40, (4, 6): 40, (4, 5): 40, (6, 5): 40, (6, 8): 40, (8, 7): 40, (8, 9): 40, (8, 10): 40, (14, 10): 40, (14, 15): 40, (7, 9): 40, (9, 21): 40, (9, 16): 40, (9, 15): 40, (9, 11): 40, (10, 11): 40, (10, 13): 40, (21, 16): 40, (16, 18): 40, (16, 17): 40, (15, 13): 40, (15, 17): 40, (11, 12): 40, (13, 12): 40, (17, 18): 40}

0 links are over capacity
The links are:
